In [1]:
from IPython.core.display import HTML
with open ("../style.css", "r") as file:
    css = file.read()
HTML(css)

In [2]:
%load_ext nb_mypy

Version 1.0.5


In [3]:
from typing import TypeVar

# Term Simplification via  Rewriting

In [4]:
import string

The type `RegExp` describes regular expressions.  These are either integers, strings, or nested tuples of integers and strings.

In [5]:
RegExp = TypeVar('RegExp')
RegExp = int | str | tuple[RegExp, ...]

In [6]:
def is_variable(s: RegExp) -> bool:
    return isinstance(s, str) and s != '𝜀' and s[0] in string.ascii_uppercase

A substitution is a dictionary mapping variable names to regular expressions.

In [7]:
Subst = dict[str, RegExp]

In [8]:
def match(pattern: RegExp, term: RegExp, Substitution: Subst = {}) -> bool:
    if is_variable(pattern):
        V = pattern
        if V in Substitution:
            return match(Substitution[V], term, Substitution) # type: ignore
        else:
            Substitution[V] = term                            # type: ignore
            return True
    if isinstance(pattern, str) or isinstance(pattern, int):
        return pattern == term
    if isinstance(term, str) or isinstance(term, int):
        return False
    if len(pattern) != len(term):
        return False
    if pattern[1] != term[1]:
        return False
    n = len(pattern)
    for i in range(n):
        if not match(pattern[i], term[i], Substitution):
            return False
    return True

In [9]:
def apply(term: RegExp, Substitution: Subst) -> RegExp:
    if is_variable(term):
        V = term
        if V in Substitution:
            return Substitution[V] # type: ignore
        else:
            return V
    if isinstance(term, str) or isinstance(term, int):
        return term
    return tuple(apply(arg, Substitution) for arg in term)

In [10]:
def rewrite(term: RegExp, rule: tuple[RegExp, RegExp]) -> tuple[bool, RegExp]:
    lhs, rhs = rule
    Substitution: Subst = {}
    if match(lhs, term, Substitution):
        return True, apply(rhs, Substitution)
    else:
        return False, term

In [11]:
def simplify_once(term: RegExp, Rules: set[tuple[RegExp, RegExp]]) -> RegExp:
    if isinstance(term, str) or isinstance(term, int):
        return term
    for rule in Rules:
        flag, simple = rewrite(term, rule)
        if flag:
            return simple
    return tuple(simplify_once(arg, Rules) for arg in term)

In [12]:
def get_rules() -> set[tuple[RegExp, RegExp]]: 
    return { (('R', '+', 0), 'R'),
             ((0, '+', 'R'), 'R'),
             (('R', '+', 'R'), 'R'),
             (('𝜀', '+', ('R', '*')), ('R', '*')),
             ((('R', '*'), '+', '𝜀'), ('R', '*')),
             (('𝜀', '+', ('R', '⋅', ('R', '*'))), ('R', '*')),
             (('𝜀', '+', (('R', '*'), '⋅', 'R')), ('R', '*')),
             ((('R', '⋅', ('R', '*')), '+', '𝜀'), ('R', '*')),
             (((('R', '*'), '⋅', 'R'), '+', '𝜀'), ('R', '*')),
             (('S', '+', ('S', '⋅', 'T')), ('S', '⋅', ('𝜀', '+', 'T'))),
             (('S', '+', ('T', '⋅', 'S')), (('𝜀', '+', 'T'), '⋅', 'S')),
             ((0, '⋅', 'R'), 0),
             (('R', '⋅', 0), 0),
             (('𝜀', '⋅', 'R'), 'R'),
             (('R', '⋅', '𝜀'), 'R'),
             ((('𝜀', '+', 'R'), '⋅', ('R', '*')), ('R', '*')),
             ((('R', '+', '𝜀'), '⋅', ('R', '*')), ('R', '*')),
             ((('R', '*'), '⋅', ('R', '+', '𝜀')), ('R', '*')),
             ((('R', '*'), '⋅', ('𝜀', '+', 'R')), ('R', '*')),
             ((0, '*'), '𝜀'),
             (('𝜀', '*'), '𝜀'),
             ((('𝜀', '+', 'R'), '*'), ('R', '*')),
             ((('R', '+', '𝜀'), '*'), ('R', '*')),
             (('R', '+', ('S', '+', 'T')), (('R', '+', 'S'), '+', 'T')),
             (('R', '⋅', ('S', '⋅', 'T')), (('R', '⋅', 'S'), '⋅', 'T')),
             ((('R', '⋅', ('S', '*')), '⋅', ('𝜀', '+', 'S')), ('R', '⋅', ('S', '*')))
           }

In [13]:
def simplify(t: RegExp) -> RegExp:
    while True:
        old_t = t
        t     = simplify_once(t, get_rules())
        if t == old_t:
            return t